In [1]:
import os

# Pastikan kaggle terpasang
os.system("pip install kaggle")

# Buat direktori untuk file kaggle.json
os.system("mkdir -p ~/.kaggle")

# Pindahkan file kaggle.json ke direktori yang tepat (sesuaikan path kaggle.json di sini)
os.system("cp /path/to/kaggle.json ~/.kaggle/")

# Set permission untuk file kaggle.json
os.system("chmod 600 ~/.kaggle/kaggle.json")

# Download dataset ke /content
os.system("kaggle datasets download -d msambare/fer2013 -p /content")

# Ekstrak file ke /content
os.system("unzip /content/fer2013.zip -d /content")


0

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hiperparameter
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = 7
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
FREEZE_LAYERS = 100
EPOCHS = 25

# Augmentasi data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data Generator
train_generator = train_datagen.flow_from_directory(
    directory='/content/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    directory='/content/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze sejumlah layer awal
for layer in base_model.layers[:FREEZE_LAYERS]:
    layer.trainable = False

# Tambahkan layer custom
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Definisi model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary model
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,917,447 (11.13 MB)

 Trainable params: 2,520,903 (9.62 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Callback untuk early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Latih model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping]
)


Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 391s 801ms/step - accuracy: 0.3450 - loss: 1.6806 - val_accuracy: 0.4289 - val_loss: 1.4829
Epoch 2/25
  1/448 ━━━━━━━━━━━━━━━━━━━━ 37s 84ms/step - accuracy: 0.5156 - loss: 1.4193

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


448/448 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.5156 - loss: 1.4193 - val_accuracy: 0.2000 - val_loss: 1.8498
Epoch 3/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 407s 775ms/step - accuracy: 0.5225 - loss: 1.2584 - val_accuracy: 0.5035 - val_loss: 1.3421
Epoch 4/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.4688 - loss: 1.3389 - val_accuracy: 0.9000 - val_loss: 0.5228
Epoch 5/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 349s 766ms/step - accuracy: 0.5625 - loss: 1.1626 - val_accuracy: 0.5540 - val_loss: 1.2480
Epoch 6/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 100us/step - accuracy: 0.6406 - loss: 1.0871 - val_accuracy: 0.9000 - val_loss: 0.5234
Epoch 7/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 385s 773ms/step - accuracy: 0.5921 - loss: 1.0911 - val_accuracy: 0.5646 - val_loss: 1.1901
Epoch 8/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.5625 - loss: 1.1805 - val_accuracy: 0.9000 - val_loss: 0.4864
Epoch 9/25
448/448 ━━━━━━━━━━━━━━━━━━━━ 351s 773ms/step - accuracy: 0.6150 - loss: 1.0262 - val_

In [1]:
# Evaluasi model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Visualisasi hasil pelatihan
import matplotlib.pyplot as plt

# Akurasi
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: name 'model' is not defined